In [1]:
import torch
from torch import nn

In [2]:
data = torch.ones(size=(10,3,229,229))

In [18]:
net = nn.Sequential(nn.Conv2d(3,6,3),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(6,4,3),
                    nn.ReLU(inplace=True),
                    nn.MaxPool2d(2),
                    nn.Conv2d(4,16,5,stride=2,padding=1),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(16,3,5,stride=3,padding=2),
                    nn.ReLU(inplace=True),
                    nn.MaxPool2d(2)
                    )

In [19]:
net(data).shape
#卷积+池化操作后得到的特征图尺寸饿大小以及特征图的数量

torch.Size([10, 3, 9, 9])

In [21]:
net = net.cuda()

In [22]:
from torch_receptive_field import receptive_field

In [23]:
rfdict = receptive_field(net,(3,229,229))

------------------------------------------------------------------------------
        Layer (type)    map size      start       jump receptive_field 
        0             [229, 229]        0.5        1.0             1.0 
        1             [227, 227]        1.5        1.0             3.0 
        2             [227, 227]        1.5        1.0             3.0 
        3             [225, 225]        2.5        1.0             5.0 
        4             [225, 225]        2.5        1.0             5.0 
        5             [112, 112]        3.0        2.0             6.0 
        6               [55, 55]        5.0        4.0            14.0 
        7               [55, 55]        5.0        4.0            14.0 
        8               [19, 19]        5.0       12.0            30.0 
        9               [19, 19]        5.0       12.0            30.0 
        10                [9, 9]       11.0       24.0            42.0 


VGG16 -Sequential使用方式

In [48]:
class NewVGG16(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_ = nn.Sequential(nn.Conv2d(3,64,3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(64,64,3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(2),

                                      nn.Conv2d(64,128,3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(128,128,3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(2),

                                      nn.Conv2d(128,256,3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(256,256,3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(256,256,3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(2),

                                      nn.Conv2d(256,512,3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(512,512,3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(512,512,3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(2),

                                      nn.Conv2d(512,512,3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(512,512,3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(512,512,3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(2)
                                      )
        
        self.clf_ = nn.Sequential(
                                nn.Dropout(0.5),
                                nn.Linear(512*7*7,4096),
                                nn.ReLU(inplace=True),
                                nn.Dropout(0.5),
                                nn.Linear(4096,4096),
                                nn.ReLU(inplace=True),
                                nn.Linear(4096,1000),
                                nn.Softmax(dim=1)
        )
    def forward(self,x):
        x = self.feature_(x)
        x = x.view(-1,512*7*7)
        output = self.clf_(x)
        return output

In [49]:
data1 = torch.ones(size=(10,3,224,224))

In [50]:
net1(data1).shape

torch.Size([10, 512, 7, 7])

In [51]:
vgg = NewVGG16()


In [52]:
from torchinfo import summary

In [53]:
summary(vgg,input_size=(3,224,224))

Layer (type:depth-idx)                   Output Shape              Param #
NewVGG16                                 [1, 1000]                 --
├─Sequential: 1-1                        [512, 7, 7]               --
│    └─Conv2d: 2-1                       [64, 224, 224]            1,792
│    └─ReLU: 2-2                         [64, 224, 224]            --
│    └─Conv2d: 2-3                       [64, 224, 224]            36,928
│    └─ReLU: 2-4                         [64, 224, 224]            --
│    └─MaxPool2d: 2-5                    [64, 112, 112]            --
│    └─Conv2d: 2-6                       [128, 112, 112]           73,856
│    └─ReLU: 2-7                         [128, 112, 112]           --
│    └─Conv2d: 2-8                       [128, 112, 112]           147,584
│    └─ReLU: 2-9                         [128, 112, 112]           --
│    └─MaxPool2d: 2-10                   [128, 56, 56]             --
│    └─Conv2d: 2-11                      [256, 56, 56]             29